# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 01.10.2023

Мягкий дедлайн: 23:59MSK 15.10.2023

Жесткий дедлайн: 23:59MSK 20.10.2023

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

Итогова оценка считается как
$$
min(task_1, task_2)*0.8 + max(task_1, task_2)*0.2
$$

где task_1 и task_2 - оценки за первое и второе заданиее соответсвенно.
Также, за домашнее задание выставляется 0, если не сделано нулевое или третье задание.
### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [ ]:
!pip install wandb --upgrade --quiet
import wandb
# !wandb login
wandb.login()
# инициализируем проект
wandb.init(project="pytorch-hw01")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


mean val MSE loss on test,█▁
mean val MSE loss on train,█▁
mean val RMSE on test,█▁
mean val RMSE on trian,█▁
mean val MSE loss on test,95.69547
mean val MSE loss on train,96.54766
mean val RMSE on test,9.78241
mean val RMSE on trian,9.82587


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import random
device = "cuda" if torch.cuda.is_available() else "cpu"
from tqdm.notebook import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2023-10-14 08:41:49--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... 

In [ ]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

Мы вывели кусок данных, чтобы понять, насколько они пригодны для работы без изменений. Здесь ясно, что сомнительно дальше с такими данными работать, потому что как минимум есть отрицательные значения, которые не отмасштабированы, кроме того еще сразу бросается в глаза совсем разная размерность, где-то видим реально большие числа, а где-то 0.075. Ясно, что будем скейлить.

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

Для выполнения данного задания (и всех последующих) предобработайте данные.

1. Зафиксируйте random_seed везде где только возможно. Вам предоставлена функция для этого, однако вы можете дополнить ее своими дополнениями
2. Обучите StandertScaler и предобработайте ваши данные. В следующих заданиях можете использовать другой scaler или вообще отказаться от него


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    # Add smth here if you need...

In [ ]:
set_random_seed(42)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


model = Ridge(alpha=1.0)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
rmse_for_model = mean_squared_error(y_test, y_pred, squared=False)
rmse_for_model

Лучшая константа для RMSE это среднее, посчитаем значение метрики при нем

In [ ]:
best_rmse_metric = mean_squared_error(y_test, np.full(len(y_test), np.mean(y_train)), squared=False)
best_rmse_metric

## Задание 1. (максимум 10 баллов)

Закрепите свои знания о том, как pytorch работает с обратным распространением ошибки, проделав следующие шаги:

1. Создайте модель линейной регрессии, которая будет состоять только из одного Linear слоя.
2. Напишите цикл обучения вашей линейной регрессии. В нем реализуйте подсчет функции потерь, сделайте шаг градиентного спуска. Запрещено использовать готовые оптимизаторы и loss-функции из библиотеки pytorch. Для подсчета градиента воспользуйтесь методом backward.
3. Запустите обучение на 10 эпохах, после каждой проверяйте значение целевой метрики на тестовой выборке.
4. Выведите на экран графики метрики и значения функции потерь на тестовой и обучающей выборке.

В данном задании нет цели побить какой-то порог по метрике. Ваша задача - убедиться в том, что ваш рукописный цикл обучения работает. Для ускорения вычислений и обучения модели можете брать только срез данных, а не весь датасет.

In [ ]:
import torch.utils.data as data_utils

share = 1 # Если понадобиться тренировать только на срезе данных

X_train_torch = torch.tensor(X_train_scaled[:len(X_train_scaled) // share], dtype=torch.float32)
y_train_torch = torch.tensor(y_train[:len(y_train) // share], dtype=torch.float32)

X_test_torch = torch.tensor(X_test_scaled[:len(X_test_scaled) // share], dtype=torch.float32)
y_test_torch = torch.tensor(y_test[:len(y_test) // share], dtype=torch.float32)

w = torch.rand(X_train_torch.size()[1], requires_grad=True, device=device, dtype=torch.float32)
b = torch.rand(1, requires_grad=True, device=device, dtype=torch.float32)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    data_utils.TensorDataset(X_train_torch, y_train_torch), batch_size=8, shuffle=True, num_workers=1
)

In [ ]:
train, test, rmse, rmse_train = [], [], [], []

for epoch in range(10):
    for x_train, y_train in tqdm(train_dataloader):  # берем батч из трейн лоадера
        y_pred = x_train.to(device) @ w.t() + b
        loss = torch.mean((y_pred - y_train.to(device)) ** 2)

        loss.backward()

        w.data -= 0.0004 * w.grad
        b.data -= 0.0004 * b.grad

        w.grad.data.zero_()
        b.grad.data.zero_()

    with torch.no_grad():  # на валидации запрещаем фреймворку считать градиенты по параметрам
        def count_mse(X, y):
          y_pred = X.to(device) @ w.t() + b # делаем предсказания
          loss = torch.mean((y_pred - y.to(device)) ** 2) # считаем лосс
          return loss

        # Считаем на трейне
        train_mse = count_mse(X_train_torch, y_train_torch)
        train.append(train_mse.cpu().numpy())
        rmse_train.append(torch.sqrt(train_mse).cpu().numpy())

        # Считаем на тесте
        test_mse = count_mse(X_test_torch, y_test_torch)
        test.append(test_mse.cpu().numpy())
        rmse.append(torch.sqrt(test_mse).cpu().numpy())

        # скидываем метрики на wandb и автоматом смотрим на графики
        wandb.log(
            {
                "mean val MSE loss on train": train[-1],
                "mean val MSE loss on test": test[-1],
                "mean val RMSE on trian": rmse_train[-1],
                "mean val RMSE on test": rmse[-1],
            }
        )

        # печатаем метрики
        print(
            f"Epoch: {epoch}, train loss: {train[-1]}, test loss: {test[-1]}, train RMSE: {rmse_train[-1]}, test RMSE: {rmse[-1]}"
        )


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 8))

# Plot train and test on the first subplot
axes[0].plot(np.arange(10), train, label="Train MSE")
axes[0].plot(np.arange(10), test, label="Test MSE")
axes[0].legend()
axes[0].set_title("Train and Test MSE")

# Plot rmse and rmse_train on the second subplot
axes[1].plot(np.arange(10), rmse, label="Test RMSE")
axes[1].plot(np.arange(10), rmse_train, label="Train RMSE")
axes[1].legend()
axes[1].set_title("Train and Test RMSE")

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

## Задание 2. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из предыдущего задания.



Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться сигнатурами функций, которые приведены ниже. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).to(device)

X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test, dtype=torch.float32).to(device)

In [ ]:
train_set = data_utils.TensorDataset(X_train_torch, y_train_torch)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True, num_workers=1)

test_set = data_utils.TensorDataset(X_test_torch, y_test_torch)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=True, num_workers=1)

model = nn.Sequential(nn.Linear(X_train_torch.size()[1], 45), nn.ReLU(), nn.Linear(45, 20), nn.ReLU(), nn.Linear(20, 1))
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
criterion = nn.MSELoss()

In [ ]:
model.to(device)
wandb.watch(model);

In [ ]:
def train(model, optimizer, criterion, train_loader, test_loader):
    """
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    """
    for x_train_iter, y_train_iter in tqdm(train_loader):
        y_pred = model(x_train_iter.to(device))
        loss = criterion(y_pred, y_train_iter.to(device))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def test(model, criterion, test_loader):
    """
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    """
    predicts = []

    with torch.no_grad():
        for x_test_iter, y_test_iter in tqdm(test_loader):
            y_pred = model(x_test_iter.to(device))
            loss = criterion(y_pred, y_test_iter.to(device))
            predicts.extend(y_pred.cpu().numpy().tolist())


    return predicts


In [ ]:
assert test(model, criterion, test_loader).shape[0] == y_test.shape[0]

## Задание 3. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

### Тест 1


1.   **Архитектура**: nn.Linear(X_train_torch.size()[1], 45), nn.ReLU(), nn.Linear(45,20), nn.ReLU(), nn.Linear(20, 1)
2.   Новый пункт